# Tutorial for HPS multiple A1-LCD WTARO 

In [1]:
# load packages
import numpy as np
import pandas as pd
import sys
import os
import time
try:
    import openmm as mm
    import openmm.app as app
    import openmm.unit as unit
except ImportError:
    import simtk.openmm as mm
    import simtk.openmm.app as app
    import simtk.unit as unit

import mdtraj
try:
    import nglview
except ImportError:
    print('Please install nglview to visualize molecules in the jupyter notebooks.')

sys.path.append('../../')
from openabc.forcefields.parsers import HPSParser
from openabc.forcefields import HPSModel
from openabc.utils.helper_functions import build_straight_CA_chain, write_pdb
from openabc.utils.insert import insert_molecules

# set simulation platform
#platform_name = 'CPU'
platform_name = 'CUDA'
#platform_name = 'OpenCL'

Parse a single A1LCD WT. The sequence length is 137.

In [2]:
#sequence = 'GSMASASSSQRGRSGSGNFGGGRGGGFGGNDNFGRGGNFSGRGGFGGSRGGGGYGGSGDGYNGFGNDGSNFGGGGSYNDFGNYNNQSSNFGPMKGGNFGGRSSGGSGGGGQYFAKPRNQGGYGGSSSSSSYGSGRRF' # WT
sequence = 'GSMASASSSQRGRSGSGNSGGGRGGGFGGNDNFGRGGNSSGRGGFGGSRGGGGYGGSGDGYNGFGNDGSNSGGGGSSNDFGNYNNQSSNFGPMKGGNFGGRSSGGSGGGGQYSAKPRNQGGYGGSSSSSSSGSGRRF' # ARO-
ca_pdb = 'init_A1LCD_WT_CA.pdb'
ca_atoms = build_straight_CA_chain(sequence, r0=0.38)
write_pdb(ca_atoms, ca_pdb)
protein_parser = HPSParser(ca_pdb)

Parse molecule with default settings.


Prepare the initial configuration.

In [3]:
# insert molecules into the simulation box randomly
n_mol = 500
if not os.path.exists('start.pdb'):
    insert_molecules(ca_pdb, 'start.pdb', n_mol, box=[100, 100, 100])
else:
    # delete and create a new start.pdb
    # otherwise, there will be an error for init_coord in the next step
    os.remove('start.pdb')
    insert_molecules(ca_pdb, 'start.pdb', n_mol, box=[100, 100, 100])


Successfully inserted 500 molecules.


Use the Urry scale optimal parameter ($\mu=1$ and $\Delta=0.08$) and run the simulation. 

In [4]:
protein = HPSModel()
for i in range(n_mol):
    protein.append_mol(protein_parser)
top = app.PDBFile('start.pdb').getTopology()
init_coord = app.PDBFile('start.pdb').getPositions()
print('size of init_coord:', len(init_coord))
protein.create_system(top, box_a=100, box_b=100, box_c=100)
protein.add_protein_bonds(force_group=1)
protein.add_contacts('Urry', mu=1, delta=0.08, force_group=2)
protein.add_dh_elec(force_group=3)
temperature = 330*unit.kelvin
friction_coeff = 0.01/unit.picosecond # use smaller friction coefficient to accelerate dynamics
timestep = 10*unit.femtosecond
integrator = mm.LangevinMiddleIntegrator(temperature, friction_coeff, timestep)
protein.set_simulation(integrator, platform_name, init_coord=init_coord)
protein.simulation.minimizeEnergy()
output_interval = 1000
output_dcd = 'output_multi_A1LCD_WT.dcd'
protein.add_reporters(output_interval, output_dcd)
protein.simulation.context.setVelocitiesToTemperature(temperature)

# run simulation
start = time.time()
totaL_steps = 100000
protein.simulation.step(totaL_steps)
end = time.time()
print('Total time for %d steps: %.2f s' % (totaL_steps, end-start))

size of init_coord: 68500
Add protein bonds.
Add nonbonded contacts.
Use Urry hydropathy scale.
Scale factor mu = 1 and shift delta = 0.08.
Add Debye-Huckel electrostatic interactions.
Set Debye length as 1 nm.
Set water dielectric as 80.0.
Use platform: CUDA
Use precision: mixed
#"Step","Time (ps)","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
1000,9.999999999999831,43913.90055271793,239934.80291641445,283848.70346913236,280.8559132587775,0
2000,20.000000000000327,34245.21992486303,256764.9718638168,291010.19178867986,300.55648363275907,943
3000,30.00000000000189,32571.93821211697,263644.00311450957,296215.9413266265,308.60874024899505,935
4000,40.00000000000061,32386.25381851893,266761.63021897094,299147.8840374899,312.2580816408266,938
5000,49.99999999999862,32958.90025952418,268278.20137365116,301237.10163317533,314.03330545784837,922
6000,59.99999999999663,34855.76617720292,268405.99365150963,303261.75982871256,

In [5]:
# visualize trajectory
traj = mdtraj.load_dcd('output_multi_A1LCD_WT.dcd', top='start.pdb')
traj.xyz -= np.mean(traj.xyz, axis=1, keepdims=True) # realign to the origin
view = nglview.show_mdtraj(traj)
view

NGLWidget(max_frame=99)

Try to perform NPT simulations at the same temperature

In [6]:
pressure = 1*unit.bar
protein.system.addForce(mm.MonteCarloBarostat(pressure, temperature))
output_interval = 1000
output_dcd = 'output_multi_A1LCD_WT_NPT.dcd'
protein.add_reporters(output_interval, output_dcd)
protein.simulation.context.reinitialize(preserveState=True)
protein.simulation.step(200000)

101000,1009.9999999992265,63758.99537294345,280202.29544223077,343961.29081517423,327.9910651855239,821
#"Step","Time (ps)","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
101000,1009.9999999992265,63758.99537294345,280202.29544223077,343961.29081517423,327.9910651855239,0
102000,1019.9999999992174,63532.182843235256,280223.61850453936,343755.8013477746,328.0160249165232,809
102000,1019.9999999992174,63532.182843235256,280223.61850453936,343755.8013477746,328.0160249165232,338
103000,1029.9999999992083,63362.602676021976,280686.9264805059,344049.5291565279,328.55835051134517,808
103000,1029.9999999992083,63362.602676021976,280686.9264805059,344049.5291565279,328.55835051134517,455
104000,1039.9999999991992,63413.35168145675,282001.32398013555,345414.6756615923,330.09691976289366,807
104000,1039.9999999991992,63413.35168145675,282001.32398013555,345414.6756615923,330.09691976289366,515
105000,1049.99999999919,63325.9738

In [7]:
# visualize trajectory
traj = mdtraj.load_dcd('output_multi_A1LCD_WT_NPT.dcd', top='start.pdb')
traj.xyz -= np.mean(traj.xyz, axis=1, keepdims=True) # realign to the origin
view = nglview.show_mdtraj(traj)
view

NGLWidget(max_frame=199)